In [16]:
import numpy as np

def g(x):
    return np.tanh(x)

def g_der(x):
    return 1 - g(x) * g(x)


def centering(x):
    
    x = np.array(x)
    mean = x.mean(axis=1, keepdims=True)
    
    return x-mean

def decorrelation(x):
    
    cov = np.cov(centering(x))
    d, E = np.linalg.eigh(cov)
    D = np.diag(d)
    D_inv = np.sqrt(np.linalg.inv(D))
    
    return E, D_inv
    
def whitening(x):
    
    E, D_inv = decorrelation(x)
    x_whiten = np.dot(E, np.dot(D_inv, np.dot(E.T, x)))
    
    return x_whiten

def calculate_new_w(w, X):
    
    w_new = (X * g(np.dot(w.T, X))).mean(axis=1) - g_der(np.dot(w.T, X)).mean() * w
    w_new /= np.sqrt((w_new ** 2).sum())
    
    return w_new

def ica(X, iterations, tolerance=1e-5):
    
    X = centering(X)
    X = whitening(X)
    components_nr = X.shape[0]
    W = np.zeros((components_nr, components_nr), dtype=X.dtype)
    
    for i in range(components_nr):
        
        w = np.random.rand(components_nr)
        for j in range(iterations):
            w_new = calculate_new_w(w, X)
        
            if i >= 1:
                w_new -= np.dot(np.dot(w_new, W[:i].T), W[:i])
                
            distance = np.abs(np.abs((w * w_new).sum()) - 1)
            w = w_new
            
            if distance < tolerance:
                break  
                
        W[i, :] = w
    S = np.dot(W, X)
    
    for i in range(len(S)):
        for j in range(len(S[i])):
            S[i][j] = round(S[i][j], 3)
    
    return S

In [17]:
X = [
[82,87,94,101,107,114,115,112,112,103,94,83],   
[77,81,87,94,104,109,110,108,107,97,89,81],   
[76,73,72,82,88,92,95,97,93,88,78,75],   
[78,83,91,100,107,110,113,112,111,102,90,77],   
[93,92,98,105,104,109,109,105,111,111,101,92],   
[95,95,98,106,102,112,108,105,113,108,100,92],   
[80,83,88,96,106,117,118,115,116,106,89,78],   
[76,78,88,95,105,115,114,112,109,104,87,74],   
[88,90,93,98,97,101,99,98,111,107,100,88],   
[79,81,87,94,97,103,99,98,106,102,86,76],   
[86,87,94,96,101,109,109,105,104,103,97,89],   
[89,89,95,103,105,110,104,104,106,108,96,90]] 

X = ica(X, iterations=1000)
print(X)

[[ 0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00
  -0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00]
 [-1.130e+00 -1.135e+00 -1.134e+00  8.090e-01  8.090e-01  8.090e-01
   8.090e-01  8.090e-01  8.090e-01  8.090e-01 -1.135e+00 -1.130e+00]
 [ 1.152e+00 -7.680e-01 -7.680e-01 -8.390e-01 -8.390e-01 -8.390e-01
   1.118e+00 -8.390e-01  1.118e+00  1.118e+00  1.152e+00 -7.680e-01]
 [ 2.330e-01 -1.550e-01 -1.560e-01 -7.620e-01 -7.620e-01 -7.620e-01
  -1.530e-01  2.745e+00 -1.530e-01 -1.530e-01  2.340e-01 -1.560e-01]
 [ 1.000e-03  5.000e-03  2.339e+00  2.000e-03  3.000e-03  3.000e-03
  -1.000e-03  1.000e-03 -1.000e-03 -1.000e-03  2.000e-03 -2.352e+00]
 [-2.346e+00 -1.000e-03 -1.000e-03  1.000e-03  1.000e-03  1.000e-03
   1.000e-03  0.000e+00  1.000e-03  1.000e-03  2.344e+00 -1.000e-03]
 [ 2.000e-03 -1.000e-03 -1.000e-03 -4.000e-03  2.349e+00 -2.342e+00
  -1.000e-03 -0.000e+00 -1.000e-03 -1.000e-03  2.000e-03 -1.000e-03]
 [ 2.480e-01 -2.652e+00  1.085e+00  1.460